In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import os
from sklearn.preprocessing import StandardScaler

# own Modules 
from nn_model import NeuralNetwork
from data_set import DataSetLoans
from trainer import Trainer
from logger import Logger

## Parameters

In [2]:
param = {
    "data" : {
        "path" : '../../data/reduced_dataset.csv' ,
    },
    "model" : {
        "input_size" : 125,
        "batch_size" : 8,
        "dropout_rate_fc": 0.2,
        "units_fc1": 40,
        "units_fc2": 20,
        "units_fc3": 10,
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.001, 
        "max_lr" :0.05
    },
    "training": {
        "n_epochs" : 50,
        "patience" : 10,
    },
    "filed_location": {
        "trained_model" : "./models/trained_model/",
        "log_file" : "./models/log/",
    },
}

# Preprocessing

In [3]:
data_set = pd.read_csv("../../data/reduced_dataset.csv")
data_set.shape

(421096, 76)

In [4]:
data_set = data_set.dropna()
data_set.shape

(326529, 76)

### Create Target Variable and Delet Samples with Status "Current"
Bei allen Loans mit dem Status Current ist nicht klar, ob diese zu einem guten oder schlechten Loan entwicklen. Deshalb lösche ich diese Samples.

In [5]:
def create_target_variable(attribute):
    if attribute == 'Fully Paid' or attribute == 'In Grace Period':
        return 0
    elif attribute == 'Current':
        return -1
    else:
        return 1
data_set['target'] = data_set['loan_status'].apply(create_target_variable)
data_set = data_set[data_set['target'] != -1]
data_set.drop(labels='loan_status', axis=1, inplace=True)
data_set.shape

(166731, 76)

### Reduce Purpose Feature

In [6]:
def new_purpose_attributes(attribute):
    if attribute=='debt_consolidation' or attribute=='credit_card':
        return 'refinance'
    elif attribute=='house' or attribute=='home_improvement' or attribute=='moving':
        return 'house'
    elif attribute=='car' or attribute=='major_purchase':
        return 'major_purchase'
    else:
        return 'other'
data_set['purpose'] = data_set['purpose'].apply(new_purpose_attributes)

### Drop Grade Feature
Ich schließe die Noten des Lending Clubs  aus. Sie werden wahrscheinlich mit Hilfe eines Clustering-Algorithmus generiert und würde einen künstlichen Einblick in die Kreditnehmer gewähren (Information Leakage) 

In [7]:
data_set.drop(labels='grade', axis=1, inplace=True)
data_set.shape

(166731, 75)

### Create Train Data Set and Target Variable

In [8]:
y = data_set['target']
X = data_set.drop(['target'], axis=1)

### Zero-Hot Encoding

In [9]:
X = pd.get_dummies(X).to_numpy()
y = pd.get_dummies(y).to_numpy()

### Create Test and Train Data Set

In [10]:
X_train_total, X_test, y_train_total, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

### Split Train Data Set in Train and Validation Set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_total, y_train_total, test_size=0.2, random_state=1, stratify=y)

### Standardize Features 

In [11]:
# Fit StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
# Transform data
train_scaled = scaler.transform(X_train)
val_scaled = scaler.transform(X_val)
test_scaled = scaler.transform(X_test)

# Prepare Training
### Create Data Set

In [13]:
dataset_train = DataSetLoans(train_scaled, y_train)
dataset_val = DataSetLoans(val_scaled, y_val)

### Initialize DataLoader

In [14]:
data_loader_training = DataLoader(dataset_train, 
                                  batch_size=param["model"]["batch_size"], 
                                  num_workers=0, 
                                  shuffle=True, 
                                  drop_last=True
                                 )
data_loader_val = DataLoader(dataset_val, 
                             batch_size=param["model"]["batch_size"], 
                             num_workers=0, 
                             shuffle=True, 
                             drop_last=True
                             )

In [15]:
for batch_idx, data in enumerate(data_loader_training):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([8, 125])
Size of target data: torch.Size([8, 2])
Data of batch: 1
Size of input data: torch.Size([8, 125])
Size of target data: torch.Size([8, 2])


### Initialize Neural Network

In [15]:
torch.manual_seed(0)
model = NeuralNetwork(batch_size=param['model']['batch_size'], 
                      input_dim=param['model']['input_size'], 
                      units_fc1=param['model']['units_fc1'], 
                      units_fc2=param['model']['units_fc1'], 
                      units_fc3=param['model']['units_fc1'], 
                      dropout_rate_fc= param['model']['dropout_rate_fc'],
                     )

### Define Binary Cross Entropy Loss function

In [16]:
criterion = nn.BCELoss()

### Initialize Optimizer and Cyclic Learning Rate Scheduler

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr=1.)  
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                              base_lr=param['cycling_lr']['base_lr'], 
                                              max_lr=param['cycling_lr']['max_lr'], 
                                              step_size_up=(len(X_train)/param['model']['batch_size'])*2, # Authors of Cyclic LR suggest setting step_size 2-8 x training iterations in epoch.
                                              mode=param['cycling_lr']['mode'],
                                              gamma=param['cycling_lr']['gamma']
                                             )

### Initialize Trainer

In [22]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  scheduler=scheduler,
                  scheduler_active = param["cycling_lr"]["scheduler_active"],
                  criterion=criterion, 
                  location_model=param["filed_location"]["trained_model"], 
                  patience=param['training']['patience']
                 )

# Start Training with Validation

In [24]:
# Initialise Logger
session_id = str(randint(10000, 99999))
logger = Logger(param["filed_location"]["log_file"], session_id)

# Log model architecture and training configuration
logger.log_message("Architecture and Training configuration:")
logger.log_message("Batch size: {}".format(param['model']['batch_size']))
logger.log_message("Input size: {}".format(param['model']['input_size']))
logger.log_message("Number of Layers: {}".format(3))
logger.log_message("Units Layer 1: {}".format(param['model']['units_fc1']))
logger.log_message("Units Layer 2: {}".format(param['model']['units_fc2']))
logger.log_message("Units Layer 3: {}".format(param['model']['units_fc3']))

logger.log_message("Dropout rate fc NN: {}".format(param['model']['dropout_rate_fc']))
logger.log_message("Cycling LR mode: {}".format(param['cycling_lr']['mode']))
logger.log_message("Cycling LR base LR: {}".format(param['cycling_lr']['base_lr']))
logger.log_message("Cycling LR max LR: {}".format(param['cycling_lr']['max_lr']))
logger.log_message("- -"*20)

print("Training phase is started")
logger.log_message("Training phase is started")
torch.manual_seed(0)

for epoch in range(param['training']['n_epochs']):
    # Train with batches 
    mean_epoch_training_loss = trainer.train(data_loader_training)
    mean_epoch_test_loss = trainer.evaluate(data_loader_val)

    # Save model if its the best one since the last change in configuration of hyperparameters
    status_ok = trainer.save_model(epoch, mean_epoch_test_loss, session_id)
    
    # Log information of current epoch
    logger.log_current_statistics(epoch, mean_epoch_test_loss)
    
    if not status_ok:
        break

print("Training phase is finished")
logger.log_message("Training phase is finished")

Training phase is started
Epoch 0: best model saved with loss: 0.06340419751046626
Epoch 1: best model saved with loss: 0.06138142402143897
Epoch 2: best model saved with loss: 0.0605426452850793
Epoch 5: best model saved with loss: 0.06042900371103321


KeyboardInterrupt: 

# Predictions from Test Set
### Load trained model

In [ ]:
checkpoint = torch.load(param["filed_location"]["trained_model"])
model.load_state_dict(checkpoint['model_state_dict'])

### Create Data Set

In [13]:
dataset_test = DataSetLoans(test_scaled, y_test)

### Initialize DataLoader

In [14]:
data_loader_test = DataLoader(dataset_test, 
                              batch_size=param["model"]["batch_size"], 
                              num_workers=0, 
                              shuffle=True, 
                              drop_last=True
                             )

### Define Binary Cross Entropy Loss function

In [16]:
criterion = nn.BCELoss()

### Initialize Optimizer and Cyclic Learning Rate Scheduler

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr=1.)  
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                              base_lr=param['cycling_lr']['base_lr'], 
                                              max_lr=param['cycling_lr']['max_lr'], 
                                              step_size_up=(len(X_train)/param['model']['batch_size'])*2, # Authors of Cyclic LR suggest setting step_size 2-8 x training iterations in epoch.
                                              mode=param['cycling_lr']['mode'],
                                              gamma=param['cycling_lr']['gamma']
                                             )

### Predict

In [ ]:
with torch.no_grad():
    model.eval()
    output = model(input_data)


# Evaluation Results

In [17]:
# Evaluate Test Set
print("The F1 Score is: {}".format(f1_score(y_test, y_hat, average="binary")))
print("The Precision Score is: {}".format(precision_score(y_test, y_hat, average="binary")))
print("The Recall Score is: {}".format(recall_score(y_test, y_hat, average="binary")))
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
print("TN: {}".format(tn))
print("FP: {}".format(fp))
print("FN: {}".format(fn))
print("TP: {}".format(tp))

The F1 Score is: 0.9424566961672235
The Precision Score is: 0.9322160910143743
The Recall Score is: 0.952924791086351
TN: 34665
FP: 995
FN: 676
TP: 13684
